In [1]:
import sys

sys.path.append('../../')
from radarange_orchestrator import llm

In [2]:
from pydantic import BaseModel, RootModel


class Article(BaseModel):
    """Reference information, including authors, title, journal and date"""

    authors: str
    title: str
    journal: str
    issue_number_page: str | None = None
    year: int | None = None


class ArticleList(RootModel[list[Article]]):
    """References list of a science article"""

    root: list[Article]

In [3]:
schema = {
    'type': 'array',
    'items': {
        'type': 'object',
        'properties': {
            'authors': {'type': 'string'},
            'title': {'type': 'string'},
            'journal': {'type': 'string'},
            'issue_number_page': {'type': 'integer'},
            'year': {'type': 'integer'},
        },
        'required': ['authors', 'title', 'journal'],
    },
}

In [4]:
with open('downloads/links.txt', 'r') as fin:
    text = fin.read()

prompt = f"""
Fill the references info from this article:
{text}
"""

prompt = """Fill the references info from this article:
[1] J.  M.  Taylor,  P.  Cappellaro,  L.  Childress,  L. 
Jiang, D. Budker, P. R. Hemmer, A. Yacoby, 
R. Walsworth, and M. D. Lukin, High-
sensitivity diamond magnetometer with 
nanoscale resolution, Nat Phys 4, 810 (2008). [2] J. R. Maze et al., Nanoscale magnetic sensing 
with an individual electronic spin in diamond, 
Nature 455, 644 (2008). 
[6] A.  Hochstetter  and  E.  Neu,  Multi-color  laser 
excitation of diamond nitrogen vacancy 
centers embedded in nanophotonic structures, 
AIP Adv 11, 65006 (2021). 
"""

# LMStudio backend

In [5]:
from radarange_orchestrator.llm_backend import LLM_Config
from radarange_orchestrator.tools import time_tool

m = llm(model='qwq-32b@q4_k_m', config=LLM_Config(ctx_size=18000, gpus=[1]))
# m = llm(model='qwen3-32b', config=LLM_Config(ctx_size=10000))
chat = m.chat(tools=[time_tool])

Connecting to host: 95.165.10.219:1234


### Auto schema

In [6]:
from radarange_orchestrator.formatting import ResponseFormat

auto_format = ResponseFormat(json_schema=ArticleList)
print(auto_format)

{
  "description": "References list of a science article",
  "items": {
    "description": "Reference information, including authors, title, journal and date",
    "properties": {
      "authors": {
        "title": "Authors",
        "type": "string"
      },
      "title": {
        "title": "Title",
        "type": "string"
      },
      "journal": {
        "title": "Journal",
        "type": "string"
      },
      "issue_number_page": {
        "anyOf": [
          {
            "type": "string"
          },
          {
            "type": "null"
          }
        ],
        "default": null,
        "title": "Issue Number Page"
      },
      "year": {
        "anyOf": [
          {
            "type": "integer"
          },
          {
            "type": "null"
          }
        ],
        "default": null,
        "title": "Year"
      }
    },
    "required": [
      "authors",
      "title",
      "journal"
    ],
    "title": "Article",
    "type": "object"
  },
  "titl

In [7]:
print(f'Prompt is {m.count_tokens(prompt)} tokens')
response = m.respond(prompt, response_format=auto_format)

Prompt is 207 tokens


In [8]:
import json

articlesj = json.loads(response.content)
print(len(articlesj))
print(json.dumps(articlesj, indent=2))

3
[
  {
    "authors": "J. M. Taylor, P. Cappellaro, L. Childress, L. Jiang, D. Budker, P. R. Hemmer, A. Yacoby, R. Walsworth, and M. D. Lukin",
    "journal": "Nat Phys",
    "title": "High-sensitivity diamond magnetometer with nanoscale resolution",
    "year": 2008
  },
  {
    "authors": "J. R. Maze et al.",
    "journal": "Nature",
    "title": "Nanoscale magnetic sensing with an individual electronic spin in diamond",
    "year": 2008
  },
  {
    "authors": "A. Hochstetter and E. Neu",
    "journal": "AIP Adv",
    "title": "Multi-color laser excitation of diamond nitrogen vacancy centers embedded in nanophotonic structures",
    "year": 2021
  }
]


### Manual schema

In [9]:
manual_format = ResponseFormat(json_schema=schema)
print(manual_format)

{
  "type": "array",
  "items": {
    "type": "object",
    "properties": {
      "authors": {
        "type": "string"
      },
      "title": {
        "type": "string"
      },
      "journal": {
        "type": "string"
      },
      "issue_number_page": {
        "type": "integer"
      },
      "year": {
        "type": "integer"
      }
    },
    "required": [
      "authors",
      "title",
      "journal"
    ]
  }
}


In [10]:
print(f'Prompt is {m.count_tokens(prompt)} tokens')
response = m.respond(prompt, response_format=manual_format)

Prompt is 207 tokens


In [11]:
articlesj = json.loads(response.content)
print(len(articlesj))
print(json.dumps(articlesj, indent=2))

3
[
  {
    "authors": "J. M. Taylor, P. Cappellaro, L. Childress, L. Jiang, D. Budker, P. R. Hemmer, A. Yacoby, R. Walsworth, and M. D. Lukin",
    "journal": "Nat Phys",
    "title": "High-sensitivity diamond magnetometer with nanoscale resolution",
    "year": 2008
  },
  {
    "authors": "J. R. Maze et al.",
    "journal": "Nature",
    "title": "Nanoscale magnetic sensing with an individual electronic spin in diamond",
    "year": 2008
  },
  {
    "authors": "A. Hochstetter and E. Neu",
    "journal": "AIP Adv",
    "title": "Multi-color laser excitation of diamond nitrogen vacancy centers embedded in nanophotonic structures",
    "year": 2021
  }
]


In [12]:
m.close()

# Llama_cpp backend

In [13]:
from radarange_orchestrator.llm_backend import LLM_Config
from radarange_orchestrator.tools import time_tool

m = llm(model='QwQ-32B-Q4_K_M.gguf', backend='llama_cpp', config=LLM_Config(ctx_size=18000, gpus=[1]))
# m = llm(model='qwen3-32b', config=LLM_Config(ctx_size=10000))
chat = m.chat(tools=[time_tool])

llama_context: n_ctx_per_seq (18000) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


### Auto format

In [14]:
print(f'Prompt is {m.count_tokens(prompt)} tokens')
response = m.respond(prompt, response_format=auto_format)

Prompt is 207 tokens


In [15]:
articlesj = json.loads(response.content)
print(len(articlesj))
print(json.dumps(articlesj, indent=2))

3
[
  {
    "authors": "J. M. Taylor, P. Cappellaro, L. Childress, L. Jiang, D. Budker, P. R. Hemmer, A. Yacoby, R. Walsworth, and M. D. Lukin",
    "title": "High-sensitivity diamond magnetometer with nanoscale resolution",
    "journal": "Nat Phys",
    "issue_number_page": "4, 810",
    "year": 2008
  },
  {
    "authors": "J. R. Maze et al.",
    "title": "Nanoscale magnetic sensing with an individual electronic spin in diamond",
    "journal": "Nature",
    "issue_number_page": "455, 644",
    "year": 2008
  },
  {
    "authors": "A. Hochstetter and E. Neu",
    "title": "Multi-color laser excitation of diamond nitrogen vacancy centers embedded in nanophotonic structures",
    "journal": "AIP Adv",
    "issue_number_page": "11, 65006",
    "year": 2021
  }
]


### Manual format

In [16]:
print(f'Prompt is {m.count_tokens(prompt)} tokens')
response = m.respond(prompt, response_format=manual_format)

Prompt is 207 tokens


In [17]:
articlesj = json.loads(response.content)
print(len(articlesj))
print(json.dumps(articlesj, indent=2))

3
[
  {
    "authors": "J. M. Taylor, P. Cappellaro, L. Childress, L. Jiang, D. Budker, P. R. Hemmer, A. Yacoby, R. Walsworth, and M. D. Lukin",
    "title": "High-sensitivity diamond magnetometer with nanoscale resolution",
    "journal": "Nat Phys",
    "issue_number_page": 810,
    "year": 2008
  },
  {
    "authors": "J. R. Maze et al.",
    "title": "Nanoscale magnetic sensing with an individual electronic spin in diamond",
    "journal": "Nature",
    "issue_number_page": 644,
    "year": 2008
  },
  {
    "authors": "A. Hochstetter and E. Neu",
    "title": "Multi-color laser excitation of diamond nitrogen vacancy centers embedded in nanophotonic structures",
    "journal": "AIP Adv",
    "issue_number_page": 65006,
    "year": 2021
  }
]


### llama_cpp killer feature: reasoning json

In [18]:
reasoning_format = ResponseFormat(json_schema=ArticleList, keep_reasoning=True)

In [19]:
print(f'Prompt is {m.count_tokens(prompt)} tokens')
response = m.respond(prompt, response_format=reasoning_format)

Prompt is 207 tokens


In [20]:
from radarange_orchestrator.chat import Chat


chat = Chat()
chat.append(response)

chat.display_thoughts()

In [21]:
m.close()